In [1]:
# IMPORTS
import nimble as nm
#nm.print_all_ipython_imports()
import glue
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
pd.options.display.mpl_style = 'default'
%matplotlib inline
from pandas.tools.plotting import scatter_matrix, scatter_plot
import seaborn as sb
import datetime as dt
from IPython.display import display
import os
from __future__ import print_function
import math
import gc
import qgrid

# Reading the files
for root,dir,files in os.walk('..\competition_data'):
    #print(root)
    print(files)

all_table_dfs = {}
for rdb_table in files:
    #print(rdb_table)
    all_table_dfs[rdb_table[:-4]] = pd.read_csv(os.path.join(root,rdb_table))
    all_table_dfs[rdb_table[:-4]].tname = rdb_table[:-4]

#print([df.tname for df in all_table_dfs.values()])
    
for df in all_table_dfs.values():
    if 'tube_assembly_id' in df.columns:
        index_col = 'tube_assembly_id'
    if 'end_form_id' in df.columns:
        index_col = 'end_form_id'
    if 'component_id' in df.columns:
        index_col = 'component_id'
    if df.tname == 'type_component':
        index_col = 'component_type_id'
    if df.tname == 'type_connection':
        index_col = 'connection_type_id'
    df.index = df[index_col]
    df.drop(index_col,1,inplace=True)
    #print(df.info())
'''
out_xlFile = pd.ExcelWriter('cat_price_data.xlsx')
for df in all_table_dfs.values():
    df.to_excel(out_xlFile,sheet_name=df.tname)
out_xlFile.save()
'''
def add_ui(df,colname,prefix='nfea_',level = 1):
    val_cts = df[colname].value_counts()
    col_ui = val_cts.sum()/val_cts
    col_ui.name = prefix + colname + '_ui' + str(level)
    df = df.join(col_ui,on=colname)
    df[col_ui.name].fillna(0,inplace=True)
    return df

['bill_of_materials.csv', 'components.csv', 'comp_adaptor.csv', 'comp_boss.csv', 'comp_elbow.csv', 'comp_float.csv', 'comp_hfl.csv', 'comp_nut.csv', 'comp_other.csv', 'comp_sleeve.csv', 'comp_straight.csv', 'comp_tee.csv', 'comp_threaded.csv', 'specs.csv', 'test_set.csv', 'train_set.csv', 'tube.csv', 'tube_end_form.csv', 'type_component.csv', 'type_connection.csv', 'type_end_form.csv']


In [2]:
# Cleaning Raw Data
nut_df = all_table_dfs['comp_nut']
tz_dict = {'M6':1.0/4,'M8':5.0/16,'M10':3.0/8,'M12':1.0/2}
for tz in tz_dict:
    nut_df.loc[nut_df.thread_size == tz,'thread_size'] = tz_dict[tz]
nut_df.thread_size = nut_df.thread_size.astype(float)
all_table_dfs['comp_nut'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 65 entries, C-1621 to C-1962
Data columns (total 10 columns):
component_type_id    65 non-null object
hex_nut_size         42 non-null float64
seat_angle           15 non-null float64
length               65 non-null float64
thread_size          65 non-null float64
thread_pitch         65 non-null float64
diameter             23 non-null float64
blind_hole           23 non-null object
orientation          65 non-null object
weight               64 non-null float64
dtypes: float64(7), object(3)
memory usage: 5.6+ KB


In [3]:
#for df in all_table_dfs.values():
#    print(df.tname+':  '+ df.index.name)
#display(all_table_dfs['train_set'].head())
masterDf = pd.concat([all_table_dfs['train_set'],all_table_dfs['test_set']])
print(masterDf.loc[masterDf.id.notnull(),'cost'].count(),all_table_dfs['train_set'].shape)
print(masterDf.loc[masterDf.id.notnull(),'cost'].count(),all_table_dfs['test_set'].shape)

0 (30213, 7)
0 (30235, 7)


## Begin FEATURE ENGINEERING

---
## Tube Assembly Related Feature Engineering

In [4]:
cat_to_num_dict = {'Yes':1,'No':0,'Y':1,'N':0}

tube_df = all_table_dfs['tube']
tube_df['nfea_material_Vol'] = math.pi*tube_df.diameter*tube_df.wall*tube_df.length
tube_df['nfea_tube_vol'] = (math.pi/4)*np.square(tube_df.diameter) * tube_df.length

end_col_list = ['end_a_1x', 'end_a_2x', 'end_x_1x', 'end_x_2x']
for col in end_col_list:
    tube_df['dfea_'+col] = tube_df[col].map(cat_to_num_dict) #pd.get_dummies(tube_df[col])['Y']

# Note that an end which is 1x shorter is always 2x shorter by default.
tube_df['nfea_nsmall_ends'] = tube_df.dfea_end_a_1x + tube_df.dfea_end_x_1x
tube_df['nfea_num_2x_ends'] = tube_df.dfea_end_a_2x + tube_df.dfea_end_x_2x
tube_df['nfea_num_1x_ends'] = tube_df.nfea_nsmall_ends - tube_df.nfea_num_2x_ends # ends which are only 1x shorter but not 2x
tube_df['dfea_high_bend_radius'] = 0
tube_df.loc[tube_df.bend_radius > 180, 'dfea_high_bend_radius'] = 1

tube_df['dfea_end_a_forming'] = tube_df.join(all_table_dfs['tube_end_form'],on='end_a')['forming'].map(cat_to_num_dict)
tube_df['dfea_end_x_forming'] = tube_df.join(all_table_dfs['tube_end_form'],on='end_x')['forming'].map(cat_to_num_dict)
tube_df['dfea_end_x_forming'] = tube_df['dfea_end_x_forming'].fillna(0)
tube_df['dfea_end_a_forming'] = tube_df['dfea_end_a_forming'].fillna(0)

tube_df['dfea_num_forming_ends'] = tube_df.dfea_end_a_forming + tube_df.dfea_end_x_forming

#experimenting...
tube_df.drop(['dfea_'+col for col in end_col_list], axis=1,inplace=True)
#tube_df.drop(['dfea_end_a_forming','dfea_end_x_forming'])

print(tube_df.columns)
tube_df = add_ui(tube_df,'material_id', level = 1)
print(tube_df.columns)
tube_df = add_ui(tube_df,'diameter', level = 1)
print(tube_df.columns)
tube_df = add_ui(tube_df,'num_bends', level = 1)


'''
material_val_counts = tube_df.material_id.value_counts()
material_ui1 = material_val_counts.sum()/material_val_counts
material_ui1.name = 'nfea_material_ui1'
tube_df = tube_df.join(material_ui1)
'''
masterDf = masterDf.join(tube_df)
#display(masterDf.head().T)
print('masterDf.columns'), print(masterDf.columns)


material_dummies = pd.get_dummies(tube_df.material_id)
print(material_dummies.columns)
tube_df[['dfea_'+col for col in material_dummies.columns]] = material_dummies

Index([u'material_id', u'diameter', u'wall', u'length', u'num_bends',
       u'bend_radius', u'end_a_1x', u'end_a_2x', u'end_x_1x', u'end_x_2x',
       u'end_a', u'end_x', u'num_boss', u'num_bracket', u'other',
       u'nfea_material_Vol', u'nfea_tube_vol', u'nfea_nsmall_ends',
       u'nfea_num_2x_ends', u'nfea_num_1x_ends', u'dfea_high_bend_radius',
       u'dfea_end_a_forming', u'dfea_end_x_forming', u'dfea_num_forming_ends'],
      dtype='object')
Index([u'material_id', u'diameter', u'wall', u'length', u'num_bends',
       u'bend_radius', u'end_a_1x', u'end_a_2x', u'end_x_1x', u'end_x_2x',
       u'end_a', u'end_x', u'num_boss', u'num_bracket', u'other',
       u'nfea_material_Vol', u'nfea_tube_vol', u'nfea_nsmall_ends',
       u'nfea_num_2x_ends', u'nfea_num_1x_ends', u'dfea_high_bend_radius',
       u'dfea_end_a_forming', u'dfea_end_x_forming', u'dfea_num_forming_ends',
       u'nfea_material_id_ui1'],
      dtype='object')
Index([u'material_id', u'diameter', u'wall', u'length', 

In [5]:
df_list = []
for i in range(1,9):
    df_list.append(all_table_dfs['bill_of_materials'][['component_id_'+str(i),'quantity_'+str(i)]])
    df_list[-1].columns = ['component_id','quantity']
tube_comps = pd.concat(df_list,axis=0)
tube_comps.fillna(value=0,inplace=True)
display(tube_comps.describe())

comp_qtys = tube_comps.groupby('component_id').sum()
comp_qtys = comp_qtys.join(all_table_dfs['components'])

,quantity
count,169584.000000
mean,0.342544
std,0.668819
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,6.000000


In [6]:
spec_df = all_table_dfs['specs'][['spec'+str(i) for i in range(1,11)]]
print('Total Spec Counts:', spec_df.count().sum())
spec_valCounts = spec_df.stack().value_counts()

rare_spec1 = spec_valCounts[spec_valCounts <= 12].index.tolist()
rare_spec2 = spec_valCounts[(spec_valCounts > 12) & (spec_valCounts < 100)].index.tolist()
pop_specs = spec_valCounts[(spec_valCounts > 100)].index.tolist()

def rep_specs(x):
    if x in rare_spec1:
        return 'rare_spec1'
    elif x in rare_spec2:
        return 'rare_spec2'
    else:
        return x
spec_df2 = spec_df.applymap(rep_specs)
display(spec_df.describe())

spec_df2.describe()
spec_type_df = pd.get_dummies(spec_df2.stack(dropna=False)).groupby(level=0).sum()
spec_type_df.rename_axis(lambda x: 'dfea_'+x,axis=1,inplace=True)
spec_type_df['nfea_spec_count'] = spec_type_df.sum(axis=1)
spec_type_df.describe()

Total Spec Counts: 29621


,spec1,spec2,spec3,spec4,spec5,spec6,spec7,spec8,spec9,spec10
count,7129,6844,5840,4154,2921,2071,535,106,20,1
unique,45,51,49,38,37,24,13,4,3,1
top,SP-0007,SP-0012,SP-0080,SP-0026,SP-0080,SP-0082,SP-0082,SP-0082,SP-0088,SP-0080
freq,3002,1736,1798,1492,1780,1167,322,73,18,1


,dfea_SP-0004,dfea_SP-0007,dfea_SP-0009,dfea_SP-0012,dfea_SP-0013,dfea_SP-0021,dfea_SP-0022,dfea_SP-0024,dfea_SP-0026,dfea_SP-0029,...,dfea_SP-0069,dfea_SP-0070,dfea_SP-0076,dfea_SP-0079,dfea_SP-0080,dfea_SP-0082,dfea_SP-0088,dfea_rare_spec1,dfea_rare_spec2,nfea_spec_count
count,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000,...,21198.000000,21198.000000,21198.000000,21198.000000,21198.00000,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000
mean,0.030758,0.146429,0.005331,0.122417,0.007784,0.005755,0.017879,0.139683,0.141995,0.005000,...,0.051043,0.052175,0.005755,0.006604,0.30267,0.110954,0.017266,0.007831,0.019813,1.397349
std,0.173482,0.356203,0.073463,0.327775,0.087884,0.075647,0.132515,0.346938,0.349053,0.070539,...,0.220090,0.222384,0.075647,0.081001,0.45963,0.314082,0.130263,0.098270,0.147901,2.198849
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.00000,0.000000,0.000000,0.000000,0.000000,3.000000
max,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,2.00000,1.000000,1.000000,4.000000,3.000000,10.000000


In [7]:
masterDf = masterDf.join(spec_type_df)

## Component Related Feature Engineering ... 

In [8]:
rdf = pd.read_excel('comp_cols_recoding.xlsx',sheetname='recoding_sheet',usecols=(0,1,2,3,4,5))

#rdf.loc[(rdf.Type == type)&(rdf.Name == colname),'Rename'].values[0]
#rdf.loc[np.all(rdf.Type == type,rdf.Name == colname),:]
#rdf.loc[rdf.Type == type,['Name','Rename']].dropna().to_dict()['Rename']
rdf.index = rdf.Name
comp_list = [tname for tname in all_table_dfs if tname.startswith('comp_')]
final_comp_dfs = []
for tname in comp_list:
    print(tname)
    re_dict = rdf.loc[(rdf.Type == tname[5:]),['Rename']].dropna().to_dict()['Rename']
    print(re_dict)
    display(all_table_dfs[tname].head())
    all_table_dfs[tname]['cweight'] = all_table_dfs[tname]['weight'] 
    all_table_dfs[tname].rename_axis(re_dict,axis=1,inplace=True)
    rel_cols = re_dict.values()
    rel_cols.append('cweight')
    final_comp_dfs.append(all_table_dfs[tname][rel_cols])

all_comps = pd.concat(final_comp_dfs)

all_comps = all_comps.join(all_table_dfs['components'],how='outer')

comp_hfl
{u'plating': u'plating', u'material': u'material_comp', u'orientation': u'orientation', u'weight': u'weight_hfl'}


,component_type_id,hose_diameter,corresponding_shell,coupling_class,material,plating,orientation,weight
component_id,,,,,,,,
C-0872,CP-023,4.8,C-0855,SP-0098,SP-0016,Yes,No,0.010
C-0873,CP-023,4.8,C-0856,SP-0098,SP-0016,Yes,No,0.010
C-0874,CP-023,4.8,C-0857,SP-0098,SP-0038,Yes,No,0.001
C-1039,CP-023,15.9,C-1040,SP-0097,SP-0095,No,No,0.052
C-1041,CP-023,15.9,C-1042,SP-0099,SP-0095,No,No,0.065


comp_nut
{u'diameter': u'nut_diameter', u'orientation': u'orientation', u'weight': u'weight_nut', u'seat_angle': u'seat_angle', u'thread_pitch': u'thread_pitch', u'length': u'nut_length', u'hex_nut_size': u'hex_size', u'thread_size': u'thread_size'}


,component_type_id,hex_nut_size,seat_angle,length,thread_size,thread_pitch,diameter,blind_hole,orientation,weight
component_id,,,,,,,,,,
C-1621,CP-025,20.64,NaN,17.0,0.687,16,NaN,NaN,No,0.015
C-1624,CP-025,34.92,NaN,26.5,1.187,12,NaN,NaN,No,0.035
C-1623,CP-025,28.58,NaN,23.5,1.000,14,NaN,NaN,No,0.044
C-1622,CP-025,23.81,NaN,20.0,0.812,16,NaN,NaN,No,0.036
C-1625,CP-025,41.28,NaN,27.5,1.437,12,NaN,NaN,No,0.129


comp_adaptor
{u'end_form_id_1': u'end_form_id_1', u'end_form_id_2': u'end_form_id_2', u'thread_size_2': u'thread_size_2', u'overall_length': u'overall_length', u'orientation': u'orientation', u'weight': u'weight_adaptor', u'nominal_size_2': u'nominal_size_2', u'nominal_size_1': u'nominal_size_1', u'unique_feature': u'unique_feature', u'thread_size_1': u'thread_size_1', u'length_1': u'length_1', u'length_2': u'length_2', u'hex_size': u'hex_size', u'connection_type_id_1': u'connection_type_id_1', u'connection_type_id_2': u'connection_type_id_2', u'thread_pitch_1': u'thread_pitch_1', u'thread_pitch_2': u'thread_pitch_2'}


...

,component_type_id,type,connection_type_id,outside_shape,base_type,height_over_tube,bolt_pattern_long,bolt_pattern_wide,groove,base_diameter,shoulder_diameter,unique_feature,orientation,weight
component_id,,,,,,,,,,,,,,
C-0008,CP-018,Boss,B-005,Round,Flat Bottom,17.0,NaN,NaN,No,22,NaN,Yes,Yes,0.032
C-0009,CP-018,Boss,B-004,Round,Flat Bottom,13.0,NaN,NaN,No,25,NaN,No,Yes,0.033
C-0020,CP-018,Boss,B-005,Round,Saddle,28.4,NaN,NaN,No,35,NaN,Yes,Yes,0.070
C-0054,CP-018,Boss,B-005,Round,Saddle,27.1,NaN,NaN,No,NaN,NaN,Yes,Yes,0.180
C-0071,CP-018,Boss,B-005,Round,Shoulder,20.0,NaN,NaN,No,30,23,Yes,Yes,0.080


comp_float
{u'thickness': u'thickness', u'weight': u'weight_float', u'bolt_pattern_wide': u'bolt_pattern_wide', u'bolt_pattern_long': u'bolt_pattern_long', u'orientation': u'orientation'}


,component_type_id,bolt_pattern_long,bolt_pattern_wide,thickness,orientation,weight
component_id,,,,,,
C-0027,CP-021,148.00,96.00,18.00,Yes,2.230
C-0454,CP-022,58.72,30.18,28.00,No,0.590
C-0455,CP-022,58.72,30.18,28.00,No,0.525
C-0494,CP-022,52.40,26.20,15.85,No,0.230
C-0496,CP-022,58.80,30.20,14.20,No,0.284


comp_tee
{u'overall_length': u'overall_length', u'orientation': u'orientation', u'extension_length': u'extension_length', u'mj_class_code': u'mj_class_code', u'unique_feature': u'unique_feature', u'bolt_pattern_wide': u'bolt_pattern_wide', u'weight': u'weight_tee', u'thickness': u'thickness', u'mj_plug_class_code': u'mj_plug_class_code', u'bolt_pattern_long': u'bolt_pattern_long', u'groove': u'groove', u'drop_length': u'drop_length'}


,component_type_id,bolt_pattern_long,bolt_pattern_wide,extension_length,overall_length,thickness,drop_length,mj_class_code,mj_plug_class_code,groove,unique_feature,orientation,weight
component_id,,,,,,,,,,,,,
C-0271,OTHER,58.70,30.20,57.10,93.0,57,28.5,MJ-003,Threaded,No,No,Yes,1.526
C-1809,OTHER,58.72,30.18,57.09,108.0,57,28.5,MJ-003,MJ-005,No,No,Yes,2.184
C-1830,OTHER,52.40,26.20,43.50,78.5,51,25.5,MJ-003,Threaded,No,Yes,Yes,1.135
C-1865,OTHER,58.70,30.20,57.10,107.0,57,28.5,MJ-003,MJ-005,No,No,Yes,1.953


In [9]:
bom_df = all_table_dfs['bill_of_materials']
s1 = bom_df[[col for col in bom_df.columns if col.startswith('component_id')]].stack(dropna=False)
s2 = bom_df[[col for col in bom_df.columns if col.startswith('quantity')]].stack(dropna=False)
s2.index = s1.index
bom_df = pd.concat([s1,s2],axis=1).dropna()
bom_df.columns=['component_id','quantity']
bom_df.head(10)

component_id  quantity
tube_assembly_id                                      
TA-00001         component_id_1       C-1622         2
                 component_id_2       C-1629         2
TA-00002         component_id_1       C-1312         2
TA-00003         component_id_1       C-1312         2
TA-00004         component_id_1       C-1312         2
TA-00005         component_id_1       C-1624         1
                 component_id_2       C-1631         1
                 component_id_3       C-1641         1
TA-00006         component_id_1       C-1624         1
                 component_id_2       C-1631         1

In [10]:
tube_master = bom_df.join(all_comps,on='component_id')

In [11]:
tempdf = bom_df.reset_index()
tempdf = tempdf.join(all_table_dfs['components'],on='component_id')
tempdf.head()

,tube_assembly_id,level_1,component_id,quantity,name,component_type_id
0,TA-00001,component_id_1,C-1622,2,NUT-SWIVEL,CP-025
1,TA-00001,component_id_2,C-1629,2,SLEEVE-ORFS,CP-024
2,TA-00002,component_id_1,C-1312,2,NUT-FLARED,CP-028
3,TA-00003,component_id_1,C-1312,2,NUT-FLARED,CP-028
4,TA-00004,component_id_1,C-1312,2,NUT-FLARED,CP-028


In [12]:
pivot_df = pd.pivot_table(tempdf,index= ['tube_assembly_id'],columns=['component_type_id'],values=['quantity'],aggfunc=[np.sum])
#tempdf.pivot(index=tempdf.tube_assembly_id, columns='component_id', values='quantity').head()

In [13]:
pivot_df.columns = ['cnt_comp_type_'+it[2] for it in pivot_df.columns]
masterDf = masterDf.join(pivot_df)
masterDf.head()

,annual_usage,bracket_pricing,cost,id,min_order_quantity,quantity,quote_date,supplier,material_id,diameter,...,cnt_comp_type_CP-021,cnt_comp_type_CP-022,cnt_comp_type_CP-023,cnt_comp_type_CP-024,cnt_comp_type_CP-025,cnt_comp_type_CP-026,cnt_comp_type_CP-027,cnt_comp_type_CP-028,cnt_comp_type_CP-029,cnt_comp_type_OTHER
tube_assembly_id,,,,,,,,,,,,,,,,,,,,,
TA-00001,0,Yes,NaN,1,0,1,2013-06-23,S-0066,SP-0035,12.7,...,NaN,NaN,NaN,2,2,NaN,NaN,NaN,NaN,NaN
TA-00001,0,Yes,NaN,2,0,2,2013-06-23,S-0066,SP-0035,12.7,...,NaN,NaN,NaN,2,2,NaN,NaN,NaN,NaN,NaN
TA-00001,0,Yes,NaN,3,0,5,2013-06-23,S-0066,SP-0035,12.7,...,NaN,NaN,NaN,2,2,NaN,NaN,NaN,NaN,NaN
TA-00001,0,Yes,NaN,4,0,10,2013-06-23,S-0066,SP-0035,12.7,...,NaN,NaN,NaN,2,2,NaN,NaN,NaN,NaN,NaN
TA-00001,0,Yes,NaN,5,0,25,2013-06-23,S-0066,SP-0035,12.7,...,NaN,NaN,NaN,2,2,NaN,NaN,NaN,NaN,NaN


### Continous Component features ....

In [14]:
#tubes_cont_final_df['nfea_tcomps_quantity'] = tube_master.quantity.groupby(level = 0).agg(np.sum)
final_cont_cols = []
continous_groups = rdf.loc[rdf.Code_List == 'continous_cols',:].dropna() #.GroupName
#print(g_cols)
for group in continous_groups.GroupName.unique():
    con_cols = np.unique(continous_groups.loc[continous_groups.GroupName == group,'Rename'].values)
    temp = tube_master[con_cols]
    tube_master[group+'_sum'] = temp.sum(axis=1)
    tube_master[group+'_mean'] = temp.mean(axis=1)
    final_cont_cols.extend([group+'_sum',group+'_mean'])
    display(group,temp.head())
    tube_master.drop(con_cols, axis=1,inplace=True)
#display(tube_master.head())

u'length'

length  length_1  length_2  length_3  \
tube_assembly_id                                                        
TA-00001         component_id_1     NaN       NaN       NaN       NaN   
                 component_id_2    9999       NaN       NaN       NaN   
TA-00002         component_id_1     NaN       NaN       NaN       NaN   
TA-00003         component_id_1     NaN       NaN       NaN       NaN   
TA-00004         component_id_1     NaN       NaN       NaN       NaN   

                                 length_4  
tube_assembly_id                           
TA-00001         component_id_1       NaN  
                 component_id_2       NaN  
TA-00002         component_id_1       NaN  
TA-00003         component_id_1       NaN  
TA-00004         component_id_1       NaN

u'nominal_size'

nominal_size_1  nominal_size_2  \
tube_assembly_id                                                  
TA-00001         component_id_1             NaN             NaN   
                 component_id_2             NaN             NaN   
TA-00002         component_id_1             NaN             NaN   
TA-00003         component_id_1             NaN             NaN   
TA-00004         component_id_1             NaN             NaN   

                                 nominal_size_3  nominal_size_4  
tube_assembly_id                                                 
TA-00001         component_id_1             NaN             NaN  
                 component_id_2             NaN             NaN  
TA-00002         component_id_1             NaN             NaN  
TA-00003         component_id_1             NaN             NaN  
TA-00004         component_id_1             NaN             NaN

u'thread_size'

thread_size  thread_size_1  thread_size_2  \
tube_assembly_id                                                             
TA-00001         component_id_1        0.812            NaN            NaN   
                 component_id_2          NaN            NaN            NaN   
TA-00002         component_id_1          NaN          0.437            NaN   
TA-00003         component_id_1          NaN          0.437            NaN   
TA-00004         component_id_1          NaN          0.437            NaN   

                                 thread_size_3  thread_size_4  
tube_assembly_id                                               
TA-00001         component_id_1            NaN            NaN  
                 component_id_2            NaN            NaN  
TA-00002         component_id_1            NaN            NaN  
TA-00003         component_id_1            NaN            NaN  
TA-00004         component_id_1            NaN            NaN

In [15]:
c_cols = np.unique(rdf.loc[(rdf.Code_List == 'continous_cols') & (rdf.GroupName.isnull()),'Rename'].dropna().values)
final_cont_cols.extend(list(c_cols))

tubes_cont_final_df = tube_master[final_cont_cols].groupby(level=0).agg([np.max,np.min,np.mean,np.sum])
tubes_cont_final_df.columns = ['_'.join(col) for col in tubes_cont_final_df.columns.values]

tubes_cont_final_df.rename_axis(lambda x: 'nfea_'+x,axis=1,inplace=True)
tubes_cont_final_df.head()

,nfea_length_sum_amax,nfea_length_sum_amin,nfea_length_sum_mean,nfea_length_sum_sum,nfea_length_mean_amax,nfea_length_mean_amin,nfea_length_mean_mean,nfea_length_mean_sum,nfea_nominal_size_sum_amax,nfea_nominal_size_sum_amin,...,nfea_weight_straight_mean,nfea_weight_straight_sum,nfea_weight_tee_amax,nfea_weight_tee_amin,nfea_weight_tee_mean,nfea_weight_tee_sum,nfea_weight_threaded_amax,nfea_weight_threaded_amin,nfea_weight_threaded_mean,nfea_weight_threaded_sum
tube_assembly_id,,,,,,,,,,,,,,,,,,,,,
TA-00001,9999,9999,9999,9999,9999,9999,9999,9999,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TA-00002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TA-00003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TA-00004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TA-00005,9999,9999,9999,9999,9999,9999,9999,9999,19.05,19.05,...,NaN,NaN,NaN,NaN,NaN,NaN,0.149,0.149,0.149,0.149


In [16]:
tube_master['tcweight'] = tube_master.cweight*tube_master.quantity

In [17]:
tubes_cont_final_df['nfea_tcweight']        = tube_master['tcweight'].groupby(level=0).agg(np.sum)
tubes_cont_final_df['nfea_tcomps_quantity'] = tube_master.quantity.groupby(level = 0).agg(np.sum)

### Dummy Component features ....

In [18]:
dummy_cols_final = []
dummy_groups = rdf.loc[rdf.Code_List == 'dummy_cols',:].dropna()
display(dummy_groups)
#pd.get_dummies(a[['binary','binary2']].stack()).sum(level=0)
for group in dummy_groups.GroupName.unique():
    grp_cols = np.unique(dummy_groups.loc[dummy_groups.GroupName == group, 'Rename'].values)
    print(grp_cols)
    print(tube_master[grp_cols].shape)
    display(tube_master[grp_cols].head(10))
    #display(tube_master[np.unique(dummy_groups.Rename.values)].stack(dropna=False))
    temp = pd.get_dummies(tube_master[grp_cols].stack(dropna=False)).sum(level=[0,1])
    temp.rename_axis(lambda x: 'dfea_'+group+'_'+x, axis=1,inplace=True)
    dummy_cols_final.extend(temp.columns)
    tube_master = tube_master.join(temp,how='outer')
    print(temp.shape)
    display(temp.head())
    tube_master.drop(grp_cols,axis=1,inplace=True)
tube_master.columns

,S.No,Type,Name,Rename,Code_List,GroupName
Name,,,,,,
connection_type_id,16,boss,connection_type_id,connection_type_id,dummy_cols,connection_type
connection_type_id,17,sleeve,connection_type_id,connection_type_id,dummy_cols,connection_type
connection_type_id_1,18,threaded,connection_type_id_1,connection_type_id_1,dummy_cols,connection_type
connection_type_id_1,19,adaptor,connection_type_id_1,connection_type_id_1,dummy_cols,connection_type
connection_type_id_2,20,threaded,connection_type_id_2,connection_type_id_2,dummy_cols,connection_type
connection_type_id_2,21,adaptor,connection_type_id_2,connection_type_id_2,dummy_cols,connection_type
connection_type_id_3,22,threaded,connection_type_id_3,connection_type_id_3,dummy_cols,connection_type
connection_type_id_4,23,threaded,connection_type_id_4,connection_type_id_4,dummy_cols,connection_type
end_form_id_1,30,threaded,end_form_id_1,end_form_id_1,dummy_cols,end_form


[u'connection_type_id' u'connection_type_id_1' u'connection_type_id_2'
 u'connection_type_id_3' u'connection_type_id_4']
(39459, 5)


connection_type_id connection_type_id_1  \
tube_assembly_id                                                          
TA-00001         component_id_1                NaN                  NaN   
                 component_id_2              B-002                  NaN   
TA-00002         component_id_1                NaN                B-007   
TA-00003         component_id_1                NaN                B-007   
TA-00004         component_id_1                NaN                B-007   
TA-00005         component_id_1                NaN                  NaN   
                 component_id_2              B-002                  NaN   
                 component_id_3                NaN                B-002   
TA-00006         component_id_1                NaN                  NaN   
                 component_id_2              B-002                  NaN   

                                connection_type_id_2 connection_type_id_3  \
tube_assembly_id                                                            
TA-00001         component_id_1                  NaN                  NaN   
                 component_id_2                  NaN                  NaN   
TA-00002         component_id_1                 9999                  NaN   
TA-00003         component_id_1                 9999                  NaN   
TA-00004         component_id_1                 9999                  NaN   
TA-00005         component_id_1                  NaN                  NaN   
                 component_id_2                  NaN                  NaN   
                 component_id_3                  NaN                  NaN   
TA-00006         component_id_1                  NaN                  NaN   
                 component_id_2                  NaN                  NaN   

                                connection_type_id_4  
tube_assembly_id                                      
TA-00001         component_id_1                  NaN  
                 component_id_2                  NaN  
TA-00002         component_id_1                  NaN  
TA-00003         component_id_1                  NaN  
TA-00004         component_id_1                  NaN  
TA-00005         component_id_1                  NaN  
                 component_id_2                  NaN  
                 component_id_3                  NaN  
TA-00006         component_id_1                  NaN  
                 component_id_2                  NaN

(39459, 14)


dfea_connection_type_9999  \
tube_assembly_id                                             
TA-00001         component_id_1                          0   
                 component_id_2                          0   
TA-00002         component_id_1                          1   
TA-00003         component_id_1                          1   
TA-00004         component_id_1                          1   

                                 dfea_connection_type_B-001  \
tube_assembly_id                                              
TA-00001         component_id_1                           0   
                 component_id_2                           0   
TA-00002         component_id_1                           0   
TA-00003         component_id_1                           0   
TA-00004         component_id_1                           0   

                                 dfea_connection_type_B-002  \
tube_assembly_id                                              
TA-00001         component_id_1                           0   
                 component_id_2                           1   
TA-00002         component_id_1                           0   
TA-00003         component_id_1                           0   
TA-00004         component_id_1                           0   

                                 dfea_connection_type_B-003  \
tube_assembly_id                                              
TA-00001         component_id_1                           0   
                 component_id_2                           0   
TA-00002         component_id_1                           0   
TA-00003         component_id_1                           0   
TA-00004         component_id_1                           0   

                                 dfea_connection_type_B-004  \
tube_assembly_id                                              
TA-00001         component_id_1                           0   
                 component_id_2                           0   
TA-00002         component_id_1                           0   
TA-00003         component_id_1                           0   
TA-00004         component_id_1                           0   

                                 dfea_connection_type_B-005  \
tube_assembly_id                                              
TA-00001         component_id_1                           0   
                 component_id_2                           0   
TA-00002         component_id_1                           0   
TA-00003         component_id_1                           0   
TA-00004         component_id_1                           0   

                                 dfea_connection_type_B-006  \
tube_assembly_id                                              
TA-00001         component_id_1                           0   
                 component_id_2                           0   
TA-00002         component_id_1                           0   
TA-00003         component_id_1                           0   
TA-00004         component_id_1                           0   

                                 dfea_connection_type_B-007  \
tube_assembly_id                                              
TA-00001         component_id_1                           0   
                 component_id_2                           0   
TA-00002         component_id_1                           1   
TA-00003         component_id_1                           1   
TA-00004         component_id_1                           1   

                                 dfea_connection_type_B-008  \
tube_assembly_id                                              
TA-00001         component_id_1                           0   
                 component_id_2                           0   
TA-00002         component_id_1                           0   
TA-00003         component_id_1                           0   
TA-00004         component_id_1                           0   

                                 dfea_connection_type_B-009  \

[u'end_form_id_1' u'end_form_id_2' u'end_form_id_3' u'end_form_id_4']
(39459, 4)


end_form_id_1 end_form_id_2 end_form_id_3  \
tube_assembly_id                                                            
TA-00001         component_id_1           NaN           NaN           NaN   
                 component_id_2           NaN           NaN           NaN   
TA-00002         component_id_1         A-002          9999           NaN   
TA-00003         component_id_1         A-002          9999           NaN   
TA-00004         component_id_1         A-002          9999           NaN   
TA-00005         component_id_1           NaN           NaN           NaN   
                 component_id_2           NaN           NaN           NaN   
                 component_id_3         A-001         A-004           NaN   
TA-00006         component_id_1           NaN           NaN           NaN   
                 component_id_2           NaN           NaN           NaN   

                                end_form_id_4  
tube_assembly_id                               
TA-00001         component_id_1           NaN  
                 component_id_2           NaN  
TA-00002         component_id_1           NaN  
TA-00003         component_id_1           NaN  
TA-00004         component_id_1           NaN  
TA-00005         component_id_1           NaN  
                 component_id_2           NaN  
                 component_id_3           NaN  
TA-00006         component_id_1           NaN  
                 component_id_2           NaN

(39459, 8)


dfea_end_form_9999  dfea_end_form_A-001  \
tube_assembly_id                                                           
TA-00001         component_id_1                   0                    0   
                 component_id_2                   0                    0   
TA-00002         component_id_1                   1                    0   
TA-00003         component_id_1                   1                    0   
TA-00004         component_id_1                   1                    0   

                                 dfea_end_form_A-002  dfea_end_form_A-003  \
tube_assembly_id                                                            
TA-00001         component_id_1                    0                    0   
                 component_id_2                    0                    0   
TA-00002         component_id_1                    1                    0   
TA-00003         component_id_1                    1                    0   
TA-00004         component_id_1                    1                    0   

                                 dfea_end_form_A-004  dfea_end_form_A-005  \
tube_assembly_id                                                            
TA-00001         component_id_1                    0                    0   
                 component_id_2                    0                    0   
TA-00002         component_id_1                    0                    0   
TA-00003         component_id_1                    0                    0   
TA-00004         component_id_1                    0                    0   

                                 dfea_end_form_A-006  dfea_end_form_A-007  
tube_assembly_id                                                           
TA-00001         component_id_1                    0                    0  
                 component_id_2                    0                    0  
TA-00002         component_id_1                    0                    0  
TA-00003         component_id_1                    0                    0  
TA-00004         component_id_1                    0                    0

Index([              u'component_id',                   u'quantity',
                    u'base_diameter',                  u'base_type',
                u'bolt_pattern_long',          u'bolt_pattern_wide',
                          u'cweight',                u'drop_length',
                      u'elbow_angle',           u'extension_length',
                           u'groove',           u'height_over_tube',
                         u'hex_size',         u'intended_nut_pitch',
              u'intended_nut_thread',              u'material_comp',
                    u'mj_class_code',         u'mj_plug_class_code',
                     u'nut_diameter',                 u'nut_length',
                      u'orientation',              u'outside_shape',
                   u'overall_length',                    u'plating',
                       u'seat_angle',          u'shoulder_diameter',
                        u'thickness',               u'thread_pitch',
                   u'thread_pitch_

In [19]:
dummy_cols = np.unique(rdf.loc[(rdf.Code_List == 'dummy_cols') & (rdf.GroupName.isnull()),'Rename'].dropna().values)

for col in ['intended_nut_pitch','seat_angle']:
    tube_master[col] = tube_master[col].astype("category", ordered=False)
    #tube_master[col] = tube_master[col].astype(str)
    #tube_master.loc[tube_master[col] == 'nan',col] = np.nan
print(tube_master.info())
#pd.get_dummies(tube_master[dummy_cols]).groupby(level=0).agg(np.sum).info()
dummy_temp = pd.get_dummies(tube_master[dummy_cols])
dummy_temp.rename_axis(lambda x: 'dfea_'+x, axis=1, inplace=True)
dummy_cols_final.extend(list(dummy_temp.columns))
tube_master = tube_master.join(dummy_temp)

tubes_dummy_final_df = tube_master[dummy_cols_final].groupby(level=0).agg(np.sum)
tubes_dummy_final_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 39459 entries, (TA-00001, component_id_1) to (TA-21197, component_id_1)
Data columns (total 76 columns):
component_id                  39459 non-null object
quantity                      39459 non-null float64
base_diameter                 417 non-null float64
base_type                     850 non-null object
bolt_pattern_long             1997 non-null float64
bolt_pattern_wide             1448 non-null float64
cweight                       39338 non-null float64
drop_length                   579 non-null float64
elbow_angle                   431 non-null float64
extension_length              578 non-null float64
groove                        3296 non-null object
height_over_tube              897 non-null float64
hex_size                      19655 non-null float64
intended_nut_pitch            12531 non-null category
intended_nut_thread           12531 non-null float64
material_comp                 16 non-null object
mj_class_code     

### Adding UI columns

In [20]:
ui_cols_final = []
ui_groups = rdf.loc[rdf.Code_List == 'ui_cols',:].dropna()
display(ui_groups)

for group in ui_groups.GroupName.unique():
    grp_cols = np.unique(ui_groups.loc[ui_groups.GroupName == group, 'Rename'].values)
    print(grp_cols)
    print(tube_master[grp_cols].shape)
    display(tube_master[grp_cols].head(10))
    temp = pd.get_dummies(tube_master[grp_cols].stack(dropna=False)).sum(level=[0,1])
    temp.rename_axis(lambda x: group+'_'+str(x), axis=1,inplace=True)
    print(temp.sum())
    print(temp.sum().sum()/temp.sum())
    temp = temp * temp.sum().sum()/temp.sum()
    temp = temp.sum(axis=1)
    temp.name = group+'_ui'
    ui_cols_final.append(temp.name)
    display(temp)
    tube_master = tube_master.join(temp,how='outer')
    tube_master.drop(grp_cols,axis=1,inplace=True)
tube_master.columns

,S.No,Type,Name,Rename,Code_List,GroupName
Name,,,,,,
thread_pitch,95,nut,thread_pitch,thread_pitch,ui_cols,thread_pitch
thread_pitch_1,96,threaded,thread_pitch_1,thread_pitch_1,ui_cols,thread_pitch
thread_pitch_1,97,adaptor,thread_pitch_1,thread_pitch_1,ui_cols,thread_pitch
thread_pitch_2,98,threaded,thread_pitch_2,thread_pitch_2,ui_cols,thread_pitch
thread_pitch_2,99,adaptor,thread_pitch_2,thread_pitch_2,ui_cols,thread_pitch
thread_pitch_3,100,threaded,thread_pitch_3,thread_pitch_3,ui_cols,thread_pitch
thread_pitch_4,101,threaded,thread_pitch_4,thread_pitch_4,ui_cols,thread_pitch


[u'thread_pitch' u'thread_pitch_1' u'thread_pitch_2' u'thread_pitch_3'
 u'thread_pitch_4']
(39459, 5)


thread_pitch  thread_pitch_1  thread_pitch_2  \
tube_assembly_id                                                                
TA-00001         component_id_1            16             NaN             NaN   
                 component_id_2           NaN             NaN             NaN   
TA-00002         component_id_1           NaN              24             NaN   
TA-00003         component_id_1           NaN              24             NaN   
TA-00004         component_id_1           NaN              24             NaN   
TA-00005         component_id_1            12             NaN             NaN   
                 component_id_2           NaN             NaN             NaN   
                 component_id_3           NaN              12             NaN   
TA-00006         component_id_1            12             NaN             NaN   
                 component_id_2           NaN             NaN             NaN   

                                 thread_pitch_3  thread_pitch_4  
tube_assembly_id                                                 
TA-00001         component_id_1             NaN             NaN  
                 component_id_2             NaN             NaN  
TA-00002         component_id_1             NaN             NaN  
TA-00003         component_id_1             NaN             NaN  
TA-00004         component_id_1             NaN             NaN  
TA-00005         component_id_1             NaN             NaN  
                 component_id_2             NaN             NaN  
                 component_id_3             NaN             NaN  
TA-00006         component_id_1             NaN             NaN  
                 component_id_2             NaN             NaN

thread_pitch_1.0          1
thread_pitch_1.25        11
thread_pitch_1.5         44
thread_pitch_1.75         3
thread_pitch_8.0          2
thread_pitch_11.5         8
thread_pitch_12.0      5829
thread_pitch_13.0         8
thread_pitch_14.0      2836
thread_pitch_16.0      7369
thread_pitch_18.0      2110
thread_pitch_20.0      1080
thread_pitch_24.0      1253
thread_pitch_27.0         4
thread_pitch_28.0         1
thread_pitch_9999.0       1
dtype: float64
thread_pitch_1.0       20560.000000
thread_pitch_1.25       1869.090909
thread_pitch_1.5         467.272727
thread_pitch_1.75       6853.333333
thread_pitch_8.0       10280.000000
thread_pitch_11.5       2570.000000
thread_pitch_12.0          3.527192
thread_pitch_13.0       2570.000000
thread_pitch_14.0          7.249647
thread_pitch_16.0          2.790066
thread_pitch_18.0          9.744076
thread_pitch_20.0         19.037037
thread_pitch_24.0         16.408619
thread_pitch_27.0       5140.000000
thread_pitch_28.0      20560.0000

tube_assembly_id                
TA-00001          component_id_1     2.790066
                  component_id_2     0.000000
TA-00002          component_id_1    16.408619
TA-00003          component_id_1    16.408619
TA-00004          component_id_1    16.408619
TA-00005          component_id_1     3.527192
                  component_id_2     0.000000
                  component_id_3     3.527192
TA-00006          component_id_1     3.527192
                  component_id_2     0.000000
                  component_id_3     3.527192
TA-00007          component_id_1     2.790066
                  component_id_2     0.000000
TA-00008          component_id_1    16.408619
TA-00009          component_id_1     3.527192
                  component_id_2     0.000000
TA-00010          component_id_1     0.000000
TA-00011          component_id_1    16.408619
TA-00012          component_id_1    16.408619
TA-00013          component_id_1     0.000000
TA-00014          component_id_1     2.790066
 

Index([                u'component_id',                     u'quantity',
                      u'base_diameter',                    u'base_type',
                  u'bolt_pattern_long',            u'bolt_pattern_wide',
                            u'cweight',                  u'drop_length',
                        u'elbow_angle',             u'extension_length',
                             u'groove',             u'height_over_tube',
                           u'hex_size',           u'intended_nut_pitch',
                u'intended_nut_thread',                u'material_comp',
                      u'mj_class_code',           u'mj_plug_class_code',
                       u'nut_diameter',                   u'nut_length',
                        u'orientation',                u'outside_shape',
                     u'overall_length',                      u'plating',
                         u'seat_angle',            u'shoulder_diameter',
                          u'thickness',            

In [21]:
ui_cols = np.unique(rdf.loc[(rdf.Code_List == 'dummy_cols') & (rdf.GroupName.isnull()),'Rename'].dropna().values)
for ui_col in ui_cols:
    temp = pd.get_dummies(tube_master[ui_col])
    temp.rename_axis(lambda x: group+'_'+str(x), axis=1,inplace=True)
    #print(temp.sum())
    #print(temp.sum().sum()/temp.sum())
    temp = temp * temp.sum().sum()/temp.sum()
    temp = temp.sum(axis=1)
    temp.name = ui_col+'_ui'
    ui_cols_final.append(temp.name)
    #display(temp.head())
    tube_master = tube_master.join(temp,how='outer')
tube_master.drop(ui_cols,axis=1,inplace=True)
tube_master.columns
tubes_ui_final_df = tube_master[ui_cols_final].groupby(level=0).agg(np.sum)
tubes_ui_final_df.rename_axis(lambda x: 'nfea_'+str(x), axis=1,inplace=True)

In [22]:
masterDf = masterDf.join(pd.concat([tubes_cont_final_df,tubes_dummy_final_df,tubes_ui_final_df],axis=1).fillna(0))

In [23]:
col_list = all_table_dfs['type_component'].index.values.tolist()
col_list.append('OTHER')
print(col_list,len(col_list))

comp_type_df = pd.DataFrame(0,index=all_table_dfs['bill_of_materials'].index,columns = col_list)
display(comp_type_df.describe())
for i in range(1,9):
    comp_joined = all_table_dfs['bill_of_materials'][['component_id_'+str(i),'quantity_'+str(i)]].join(
        all_table_dfs['components'],on='component_id_'+str(i))
    dummies = pd.get_dummies(comp_joined['component_type_id'])
    for col in dummies.columns:
        comp_type_df[col] = comp_type_df[col] + dummies[col]*all_table_dfs['bill_of_materials']['quantity_'+str(i)].fillna(0)
comp_type_df.describe()

['CP-001', 'CP-002', 'CP-003', 'CP-004', 'CP-005', 'CP-006', 'CP-007', 'CP-008', 'CP-009', 'CP-010', 'CP-011', 'CP-012', 'CP-013', 'CP-014', 'CP-015', 'CP-016', 'CP-017', 'CP-018', 'CP-019', 'CP-020', 'CP-021', 'CP-022', 'CP-023', 'CP-024', 'CP-025', 'CP-026', 'CP-027', 'CP-028', 'CP-029', 'OTHER'] 30


,CP-001,CP-002,CP-003,CP-004,CP-005,CP-006,CP-007,CP-008,CP-009,CP-010,...,CP-021,CP-022,CP-023,CP-024,CP-025,CP-026,CP-027,CP-028,CP-029,OTHER
count,21198,21198,21198,21198,21198,21198,21198,21198,21198,21198,...,21198,21198,21198,21198,21198,21198,21198,21198,21198,21198
mean,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
std,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
min,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25%,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50%,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
75%,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
max,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,CP-001,CP-002,CP-003,CP-004,CP-005,CP-006,CP-007,CP-008,CP-009,CP-010,...,CP-021,CP-022,CP-023,CP-024,CP-025,CP-026,CP-027,CP-028,CP-029,OTHER
count,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000,...,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000
mean,0.009341,0.019341,0.018351,0.020096,0.000189,0.009812,0.008774,0.011369,0.005189,0.004812,...,0.000472,0.001132,0.000755,0.941268,0.694358,0.242759,0.005567,0.160487,0.000047,0.185819
std,0.121740,0.162258,0.158403,0.140669,0.013736,0.105953,0.102423,0.119413,0.082830,0.069201,...,0.021715,0.037603,0.027464,0.885955,0.852104,0.626910,0.106261,0.539256,0.006868,0.690996
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.000000,2.000000,2.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,1.000000,...,1.000000,2.000000,1.000000,6.000000,6.000000,4.000000,5.000000,6.000000,1.000000,11.000000


In [24]:
masterDf.quote_date = pd.to_datetime(masterDf.quote_date)

#month_dummies = pd.get_dummies(masterDf.quote_date.apply(lambda x: x.month))
#masterDf[['dfea_month_'+str(col) for col in month_dummies.columns]] = month_dummies

yr_dummies = pd.get_dummies(masterDf.quote_date.apply(lambda x: x.year))

In [25]:
rel_yrs = [col for col in yr_dummies.columns if col > 2000 and col < 2016]

In [26]:
masterDf[['dfea_year_'+str(col) for col in rel_yrs]] = yr_dummies[rel_yrs]

In [27]:
masterDf.head()

,annual_usage,bracket_pricing,cost,id,min_order_quantity,quantity,quote_date,supplier,material_id,diameter,...,dfea_year_2006,dfea_year_2007,dfea_year_2008,dfea_year_2009,dfea_year_2010,dfea_year_2011,dfea_year_2012,dfea_year_2013,dfea_year_2014,dfea_year_2015
tube_assembly_id,,,,,,,,,,,,,,,,,,,,,
TA-00001,0,Yes,NaN,1,0,1,2013-06-23,S-0066,SP-0035,12.7,...,0,0,0,0,0,0,0,1,0,0
TA-00001,0,Yes,NaN,2,0,2,2013-06-23,S-0066,SP-0035,12.7,...,0,0,0,0,0,0,0,1,0,0
TA-00001,0,Yes,NaN,3,0,5,2013-06-23,S-0066,SP-0035,12.7,...,0,0,0,0,0,0,0,1,0,0
TA-00001,0,Yes,NaN,4,0,10,2013-06-23,S-0066,SP-0035,12.7,...,0,0,0,0,0,0,0,1,0,0
TA-00001,0,Yes,NaN,5,0,25,2013-06-23,S-0066,SP-0035,12.7,...,0,0,0,0,0,0,0,1,0,0


---
### Create Categorical Variables here...

In [28]:
def add_dummies(df,colname):
    col_dummies = pd.get_dummies(df[colname])
    df[['dfea_'+colname+'_'+ col for col in col_dummies.columns]] = col_dummies

In [29]:
masterDf = add_ui(masterDf,'supplier',level = 2)
#masterDf = add_ui(masterDf,'component_type_id', level = 2)
top_suppliers = masterDf.supplier.value_counts()[:30]
masterDf.supplier = masterDf.supplier.apply(lambda x: x if str(x) in top_suppliers else np.nan)
add_dummies(masterDf,'supplier')

In [30]:
del all_table_dfs, tube_comps
del tube_df
del tubes_dummy_final_df
del tubes_cont_final_df
del tubes_ui_final_df
del dummy_groups
del all_comps
del bom_df, comp_joined, c_cols, comp_qtys, comp_type_df, con_cols, continous_groups, df, dummies, dummy_cols, dummy_temp, grp_cols, material_dummies, nut_df, rdf, s1, s2, spec_type_df, spec_valCounts, temp, top_suppliers, tube_master, ui_cols, ui_groups, yr_dummies
#del month_dummies
gc.collect()
#%whos
masterDf.to_pickle('masterDf.pkl')

---

# ML Starts... 

In [2]:
masterDf = pd.read_pickle('masterDf.pkl')
masterDf['dfea_bracket_pricing'] = masterDf.bracket_pricing.map({'Yes':1,'No':0})
masterDf['nfea_ln1p_annual_usage'] = np.log1p(masterDf.annual_usage.values)
masterDf['nfea_ln1p_quantity'] = np.log1p(masterDf.quantity.values)
#masterDf['nfea_ln1p_tube_vol'] = np.log1p(masterDf.nfea_tube_vol.values)

In [3]:
# Let's drop useless features, based on the feature importances ... Experimenting Trial  ONLY
#drop_lst = [col for col in masterDf.columns if col.startswith('dfea_intended_nut_pitch')] + \
#[col for col in masterDf.columns if col.startswith('dfea_end_form')]
#masterDf.drop(drop_lst, axis=1,inplace=True)

In [4]:
#Drop Outliers
#masterDf.drop(masterDf.loc[masterDf.cost > 500,:].index,inplace=True)

In [5]:
#masterDf.cost.plot()

In [6]:
fea_cols = ['min_order_quantity','quantity','diameter','wall','length','num_bends','bend_radius',
            'other','num_bracket','num_boss','annual_usage']

def check_prefixes(col):
    for prefix in ['dfea_','nfea_','cnt_']:
        if col.startswith(prefix):
            return True
    return False

fea_cols.extend([col for col in masterDf.columns if check_prefixes(col)])

for col in fea_cols:
    masterDf[col].fillna(value=0,inplace=True)
    print(masterDf[col].isnull().sum().sum(), col)

0 min_order_quantity
0 quantity
0 diameter
0 wall
0 length
0 num_bends
0 bend_radius
0 other
0 num_bracket
0 num_boss
0 annual_usage
0 nfea_material_Vol
0 nfea_tube_vol
0 nfea_nsmall_ends
0 nfea_num_2x_ends
0 nfea_num_1x_ends
0 dfea_high_bend_radius
0 dfea_end_a_forming
0 dfea_end_x_forming
0 dfea_num_forming_ends
0 nfea_material_id_ui1
0 nfea_diameter_ui1
0 nfea_num_bends_ui1
0 dfea_SP-0004
0 dfea_SP-0007
0 dfea_SP-0009
0 dfea_SP-0012
0 dfea_SP-0013
0 dfea_SP-0021
0 dfea_SP-0022
0 dfea_SP-0024
0 dfea_SP-0026
0 dfea_SP-0029
0 dfea_SP-0047
0 dfea_SP-0050
0 dfea_SP-0057
0 dfea_SP-0058
0 dfea_SP-0061
0 dfea_SP-0063
0 dfea_SP-0067
0 dfea_SP-0068
0 dfea_SP-0069
0 dfea_SP-0070
0 dfea_SP-0076
0 dfea_SP-0079
0 dfea_SP-0080
0 dfea_SP-0082
0 dfea_SP-0088
0 dfea_rare_spec1
0 dfea_rare_spec2
0 nfea_spec_count
0 cnt_comp_type_CP-001
0 cnt_comp_type_CP-002
0 cnt_comp_type_CP-003
0 cnt_comp_type_CP-004
0 cnt_comp_type_CP-005
0 cnt_comp_type_CP-006
0 cnt_comp_type_CP-007
0 cnt_comp_type_CP-008
0 cnt_c

In [7]:
from sklearn.cross_validation import train_test_split
masterDf['target'] = np.log1p(masterDf.cost.values)
#masterDf.to_csv('masterDf.csv',date_format='%d %b %Y')
#&(masterDf.dfea_bracket_pricing >= 0)


# experimenting
#fea_cols.remove('quantity')
#fea_cols.remove('nfea_ln1p_quantity')
#fea_cols = [el for el in fea_cols if not el.startswith('dfea_year')]
#fea_cols = [el for el in fea_cols if not el.startswith('dfea_supplier')]
#fea_cols.remove('diameter')
#fea_cols.remove('nfea_material_Vol')
#fea_cols.remove('nfea_tube_vol')
#fea_cols.remove('nfea_diameter_ui1')
#fea_cols.remove('nfea_tcweight')

X = masterDf.loc[masterDf.id.isnull() &(masterDf.dfea_bracket_pricing >= 0), fea_cols].values
Y = masterDf.loc[masterDf.id.isnull() &(masterDf.dfea_bracket_pricing >= 0), 'target'].values

X_ktest   = masterDf.loc[masterDf.id.notnull()&(masterDf.dfea_bracket_pricing >= 0), fea_cols].values
result_df = masterDf.loc[masterDf.id.notnull()&(masterDf.dfea_bracket_pricing >= 0),['id','cost']]

gc.collect()
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.20)

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.grid_search import GridSearchCV

df_dict = {}

def eval_model(model, X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,df_dict=df_dict):
    for x,y,txt in [(X_train,Y_train,'TRAIN'),(X_test,Y_test,'TEST')]:
        pY = model.predict(x)
        rmsle = math.sqrt(np.mean((y - pY)**2))
        print(txt+' RMSLE: ', rmsle)
        df_dict[txt] = pd.DataFrame({'actual':y,'predicted':pY})

def learn_model(model, X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,df_dict=df_dict): # fea_cols=fea_cols
    model.fit(X_train,Y_train)
    eval_model(model, X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test)
    df_dict['fi_df'] = pd.DataFrame(fea_cols,model.feature_importances_)
    #print([fea+': '+str(imp) for fea,imp in zip(fea_cols,rf_reg.feature_importances_)],sep='\n')

def exec_gs(model, param_grid, X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test):
    gs_model = GridSearchCV(model,param_grid, n_jobs = 1)
    gs_model.fit(X_train,Y_train)
    print(gs_model.best_params_)
    eval_model(gs_model, X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test)
    return gs_model.best_estimator_

def gen_submission(model,fname,X=X,Y=Y,X_ktest=X_ktest,result_df=result_df):
    model.fit(X,Y)
    Y_ktest = model.predict(X_ktest)
    result_df.cost = np.exp(Y_ktest) - 1
    result_df.loc[:,'id'] = result_df.loc[:,'id'].astype(int)
    result_df.to_csv(fname, index=False)

In [11]:
rf_reg = RandomForestRegressor(n_estimators=30,max_depth=20,bootstrap=True,verbose=1,n_jobs=-1)
learn_model(rf_reg)
#rf_gs_reg = exec_gs(RandomForestRegressor(verbose=1),param_grid= {'n_estimators':[50,100,200,500]})

[Parallel(n_jobs=-1)]: Done   1 out of  30 | elapsed:    2.5s remaining:  1.2min
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   19.9s finished
[Parallel(n_jobs=4)]: Done   1 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:    0.0s finished


TRAIN RMSLE:  0.100661676901
TEST RMSLE:  0.231873300589


[Parallel(n_jobs=4)]: Done   1 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:    0.0s finished


In [ ]:
#df_dict['TEST'].to_clipboard()
#df_dict['TRAIN'].to_clipboard()
df_dict['fi_df'].to_clipboard()

In [12]:
pred_rf_X_train = rf_reg.predict(X_train)
pred_rf_X_test = rf_reg.predict(X_test)

#print(pred_rf.reshape(pred_rf.size,1).shape)
#print(X_train.shape)
#print(np.append(X_train,pred_rf.reshape(pred_rf.size,1),axis=1).shape)
X_train2 = np.append(X_train,pred_rf_X_train.reshape(pred_rf_X_train.size,1),axis=1)
X_test2 = np.append(X_test,pred_rf_X_test.reshape(pred_rf_X_test.size,1),axis=1)

[Parallel(n_jobs=4)]: Done   1 out of  11 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   1 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:    0.0s finished


In [321]:
et_reg = ExtraTreesRegressor(verbose=1,n_estimators=30, max_depth=20,bootstrap=True,n_jobs=-1) #,criterion='entropy'
# max_depth=20,

#et_reg = ExtraTreesRegressor(verbose=1,n_estimators=30, max_depth=13,bootstrap=True,n_jobs=-1)

#gs_et = GridSearchCV(et_reg,param_grid={'n_estimators':[5,10,15],'max_depth':[1,3,5,7,9,10,13,15,18]})
learn_model(et_reg,X_train = X_train2,X_test=X_test2)

#for mdep in [5,7,9,10,13,15,18]:
#    et_regtest = ExtraTreesRegressor(verbose=1,n_estimators=28, max_depth=mdep,bootstrap=True,n_jobs=-1) 
#    learn_model(et_regtest)

[Parallel(n_jobs=-1)]: Done   1 out of  30 | elapsed:    2.0s remaining:  1.0min
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   16.5s finished
[Parallel(n_jobs=4)]: Done   1 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:    0.0s finished


TRAIN RMSLE:  0.0458552742088
TEST RMSLE:  0.222325540128


[Parallel(n_jobs=4)]: Done   1 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:    0.0s finished


ValueError: Shape of passed values is (1, 301), indices imply (1, 302)

In [39]:
gen_submission(rf_reg,'rf_reg_submit_'+dt.date.today().strftime('%d%b')+'.csv')

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    1.5s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:  1.3min
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  2.5min finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.0s finished


In [40]:
gen_submission(et_reg,'et_reg_submit_'+dt.date.today().strftime('%d%b')+'.csv')

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    1.0s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:   59.0s
[Parallel(n_jobs=1)]: Done 200 jobs       | elapsed:  4.1min
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  6.1min finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done 200 jobs       | elapsed:    2.4s
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    3.7s finished


In [331]:
ab_reg = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=13), n_estimators=30,loss='linear') #exponential

#ab_reg = AdaBoostRegressor(base_estimator=RandomForestRegressor(n_estimators=15), n_estimators=10,loss='linear') #exponential

learn_model(ab_reg)

TRAIN RMSLE:  0.101201800833
TEST RMSLE:  0.229330359586


In [333]:
gen_submission(ab_reg,'ab_reg_submit_'+dt.date.today().strftime('%d%b')+'.csv')

In [325]:
gb_reg = GradientBoostingRegressor(n_estimators=50,verbose=1, max_depth=13,subsample=0.5,learning_rate=0.1)
learn_model(gb_reg)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.5517           0.1159            6.89m
         2           0.4625           0.0938            5.75m
         3           0.3864           0.0751            5.41m
         4           0.3208           0.0612            4.99m
         5           0.2660           0.0497            4.76m
         6           0.2171           0.0408            4.58m
         7           0.1851           0.0337            4.47m
         8           0.1514           0.0276            4.36m
         9           0.1339           0.0214            4.20m
        10           0.1130           0.0183            4.10m
        20           0.0301           0.0026            2.96m
        30           0.0135           0.0003            1.88m
        40           0.0090           0.0001           53.34s
        50           0.0067           0.0000            0.00s
TRAIN RMSLE:  0.0841117036184
TEST RMSLE:  0.206404380191


In [332]:
gen_submission(gb_reg,'gb_reg_submit_'+dt.date.today().strftime('%d%b')+'.csv')

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.5588           0.1142            6.93m
         2           0.4571           0.0934            6.40m
         3           0.3741           0.0761            6.36m
         4           0.3094           0.0625            6.23m
         5           0.2570           0.0508            6.03m
         6           0.2170           0.0403            5.98m
         7           0.1810           0.0331            5.84m
         8           0.1532           0.0271            5.66m
         9           0.1324           0.0219            5.50m
        10           0.1110           0.0177            5.35m
        20           0.0293           0.0024            3.85m
        30           0.0146           0.0003            2.45m
        40           0.0099           0.0001            1.15m
        50           0.0072           0.0000            0.00s


In [324]:
learn_model(gb_reg,X_train=X_train2,X_test=X_test2)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.5475           0.1300            6.64m
         2           0.4474           0.1047            5.91m
         3           0.3658           0.0845            5.51m
         4           0.2919           0.0695            5.25m
         5           0.2428           0.0550            5.01m
         6           0.1977           0.0445            4.87m
         7           0.1615           0.0358            4.71m
         8           0.1296           0.0297            4.54m
         9           0.1061           0.0238            4.38m
        10           0.0905           0.0186            4.23m
        20           0.0130           0.0024            2.92m
        30           0.0034           0.0003            1.82m
        40           0.0018           0.0000           51.81s
        50           0.0014          -0.0000            0.00s
TRAIN RMSLE:  0.038376657029
TEST RMSLE:  0.222058583953


ValueError: Shape of passed values is (1, 301), indices imply (1, 302)

Trying Xgboost

In [13]:
import sys
#sys.path.append('D:\PDA_Learning\aa_nimble_devel\py_ml_utils-master')
sys.path.append("C:\\Python27_x64\\xgboost-master\\windows\\x64\\Release")
import xgboost as xgb

In [19]:
# fit a random forest model
params = {}
params["objective"] = "reg:linear"
params["eta"] = 0.05
params["min_child_weight"] = 5
params["subsample"] = 0.8
params["colsample_bytree"] = 0.8
params["scale_pos_weight"] = 1.0
params["silent"] = 1
params["max_depth"] = 7


#X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.20)
'''
for col in fea_cols:
    masterDf[col] = masterDf[col].astype(float)

test  = masterDf.loc[masterDf.id.notnull(),fea_cols].values

masterDf['target'] = masterDf['target'].astype(float)
labels = masterDf.loc[masterDf.id.isnull(),'target'].values

train = masterDf.loc[masterDf.id.isnull(),fea_cols].values
'''
plst = list(params.items())


num_rounds = 3000
xgtrain = xgb.DMatrix(X_train2.astype(float), label=Y_train.astype(float))
XGB_model = xgb.train(plst, xgtrain, num_rounds)

In [20]:
eval_model(model=XGB_model,X_test=xgb.DMatrix(X_test2.astype(float)),X_train=xgb.DMatrix(X_train2.astype(float)))

TRAIN RMSLE:  0.0211708696994
TEST RMSLE:  0.217190977816


In [84]:
def gen_xgb_submission(fname,X=X,Y=Y,X_ktest=X_ktest,result_df=result_df,plst=plst,num_rounds=num_rounds):
    model_final = xgb.train(plst,xgb.DMatrix(X.astype(float), label=Y.astype(float)),num_rounds)
    Y_ktest = model.predict(xgb.DMatrix(X_ktest.astype(float)))
    result_df.cost = np.exp(Y_ktest) - 1
    result_df.loc[:,'id'] = result_df.loc[:,'id'].astype(int)
    result_df.to_csv(fname, index=False)    

In [86]:
gen_xgb_submission('xgb_reg_submit_4000_'+dt.date.today().strftime('%d%b')+'.csv')